In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data18 = data.loc[data['Year']== '2018']
data18 = data18.loc[data18['Genre1']== 'Electronic']
data18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
16,17,Lover Chanting EP,Little Dragon,have long seen their sounds as a way out of r...,Ninja Tune,5.2,November,19,2018,Electronic,NaN,NaN,NaN
88,89,Ens,BEAST,"to paraphrase ,, listening to ,—the first full...",Thrill Jockey,6.8,December,1,2018,Electronic,NaN,NaN,NaN
100,101,81b EP,Joy Orbison,peter o’grady has not had the career that most...,Hinge Finger,7.7,October,26,2018,Electronic,NaN,NaN,NaN
101,102,From the Dirt,Waajeed,hip-hop and house are the gallagher brothers o...,Dirt Tech Reck,7.4,December,5,2018,Electronic,NaN,NaN,NaN
117,118,Homework,Daft Punk,"’s , is, in its pure existence, a study in con...",Virgin,9.2,December,2,2018,Electronic,NaN,NaN,NaN
122,123,Breakfast,The Samps,if the name mötley crüe (give or take some let...,Gloriette,7.3,December,5,2018,Electronic,NaN,NaN,NaN
140,141,Terminal,ZULI,"in 2015, cairo producer ahmed el ghazoly ,: “d...",UIQ,7.7,November,28,2018,Electronic,NaN,NaN,NaN
141,142,Daily Routine,Peder Mannerfelt,has had an exceptionally busy decade. in addi...,Peder Mannerfelt Produktion,7.9,November,28,2018,Electronic,NaN,NaN,NaN
148,150,Fever Focus EP,Jacques Greene,"this past summer, , launched his party series ...",LuckyMe,7.7,November,27,2018,Electronic,NaN,NaN,NaN


In [5]:
data.Genre1.unique()

array(['Pop/R&B', 'Rock', 'Experimental', 'Rap', 'Electronic',
       'Folk/Country', 'Global', 'Metal', 'Jazz'], dtype=object)

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data18['content']))

print(data_words[:1])

[['long', 'musical', 'journey', 'has', 'come', 'full', 'circle', 'back', 'in', 'the', 'he', 'and', 'taylor', 'started', 'making', 'music', 'together', 'in', 'the', 'southern', 'california', 'hardcore', 'act', 'ex', 'ignota', 'then', 'traipsed', 'up', 'to', 'san', 'francisco', 'to', 'start', 'band', 'called', 'whose', 'change', 'of', 'musical', 'style', 'was', 'signaled', 'by', 'the', 'fact', 'that', 'they', 'borrowed', 'their', 'name', 'from', 'one', 'of', 'most', 'popular', 'albums', 'upon', 'moving', 'to', 'north', 'carolina', 'in', 'the', 'late', 'the', 'duo', 'made', 'music', 'as', 'with', 'taylor', 'the', 'singer', 'and', 'chief', 'songwriter', 'and', 'hirsch', 'multi', 'instrumentalist', 'arranger', 'and', 'producer', 'following', 'brief', 'sojourn', 'in', 'brooklyn', 'hirsch', 'moved', 'back', 'to', 'the', 'west', 'coast', 'in', 'settling', 'not', 'too', 'far', 'from', 'where', 'he', 'and', 'taylor', 'first', 'started', 'making', 'music', 'together', 'that', 'transitory', 'life'

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['long', 'musical', 'journey', 'has', 'come', 'full', 'circle', 'back', 'in', 'the', 'he', 'and', 'taylor', 'started', 'making', 'music', 'together', 'in', 'the', 'southern', 'california', 'hardcore', 'act', 'ex', 'ignota', 'then', 'traipsed', 'up', 'to', 'san', 'francisco', 'to', 'start', 'band', 'called', 'whose', 'change', 'of', 'musical', 'style', 'was', 'signaled', 'by', 'the', 'fact', 'that', 'they', 'borrowed', 'their', 'name', 'from', 'one', 'of', 'most', 'popular', 'albums', 'upon', 'moving', 'to', 'north', 'carolina', 'in', 'the', 'late', 'the', 'duo', 'made', 'music', 'as', 'with', 'taylor', 'the', 'singer', 'and', 'chief', 'songwriter', 'and', 'hirsch', 'multi', 'instrumentalist', 'arranger', 'and', 'producer', 'following', 'brief', 'sojourn', 'in', 'brooklyn', 'hirsch', 'moved', 'back', 'to', 'the', 'west', 'coast', 'in', 'settling', 'not', 'too', 'far', 'from', 'where', 'he', 'and', 'taylor', 'first', 'started', 'making', 'music', 'together', 'that', 'transitory', 'life',

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['long', 'musical', 'journey', 'come', 'full', 'circle', 'back', 'taylor', 'start', 'make', 'music', 'together', 'southern', 'california', 'hardcore', 'act', 'ignota', 'traipse', 'start', 'band', 'call', 'whose', 'change', 'musical', 'style', 'signal', 'fact', 'borrow', 'name', 'popular', 'album', 'move', 'north', 'carolina', 'late', 'duo', 'make', 'music', 'taylor', 'singer', 'chief', 'songwriter', 'multi', 'instrumentalist', 'arranger', 'producer', 'follow', 'brief', 'sojourn', 'move', 'back', 'west', 'coast', 'settle', 'far', 'first', 'start', 'make', 'music', 'together', 'transitory', 'life', 'general', 'travail', 'tour', 'musician', 'provide', 'grist', 'hirsch', 'solo', 'debut', 'release', 'move', 'country', 'primary', 'lyrically', 'musically', 'write', 'movingly', 'emotional', 'geographic', 'transience', 'borrow', 'array', 'country', 'source', 'create', 'easygoing', 'yet', 'endlessly', 'inventive', 'deeply', 'soulful', 'folk', 'rock', 'sound', 'picaresque', 'american', 'landscap

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 3), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1)]]


In [11]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.052*"party" + 0.038*"weird" + 0.034*"young" + 0.031*"hook" + '
  '0.024*"stand" + 0.020*"soundtrack" + 0.019*"color" + 0.019*"question" + '
  '0.018*"later" + 0.018*"list"'),
 (1,
  '0.022*"exactly" + 0.022*"modern" + 0.019*"history" + 0.019*"rock" + '
  '0.015*"friend" + 0.014*"box" + 0.013*"clear" + 0.012*"pair" + 0.012*"cover" '
  '+ 0.012*"repeat"'),
 (2,
  '0.047*"energy" + 0.044*"tell" + 0.032*"lyric" + 0.032*"percussion" + '
  '0.031*"output" + 0.029*"edge" + 0.029*"cover" + 0.025*"stay" + '
  '0.023*"system" + 0.023*"post"'),
 (3,
  '0.011*"era" + 0.010*"digital" + 0.009*"funk" + 0.009*"side" + 0.009*"drone" '
  '+ 0.008*"quality" + 0.008*"rise" + 0.008*"breakbeat" + 0.008*"edit" + '
  '0.007*"add"'),
 (4,
  '0.070*"culture" + 0.057*"future" + 0.057*"electro" + 0.039*"analog" + '
  '0.038*"full_length" + 0.035*"throb" + 0.032*"pad" + 0.032*"remix" + '
  '0.031*"post" + 0.028*"room"'),
 (5,
  '0.034*"disco" + 0.023*"lyric" + 0.023*"man" + 0.021*"word" + 0.018*"people" 

In [13]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.511619517384246

Coherence Score:  0.2103937692845709


In [14]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.016358  0.031958       1        1  24.230278
14     0.023217 -0.036796       2        1  15.595338
1      0.015096  0.051484       3        1  10.062228
7     -0.044694 -0.133234       4        1   8.266759
5     -0.030948  0.008645       5        1   6.861267
18    -0.074127  0.011867       6        1   5.251218
9     -0.019481  0.032581       7        1   4.570961
10    -0.122327  0.175637       8        1   3.747361
11    -0.107565 -0.044449       9        1   3.432938
0     -0.071245  0.165429      10        1   3.270198
15     0.056136 -0.164830      11        1   2.776208
19    -0.179860 -0.137659      12        1   2.637221
13     0.142194 -0.048339      13        1   2.397185
12     0.013499 -0.017361      14        1   2.309562
2      0.051494 -0.047488      15        1   1.467304
6     -0.002045  0.086424      16        1   0.941014
4      0.246342  0.084385      17        1   0.933128
17     0.061655 -0.058729      18        1   0.822233
8      0.026151  0.035469      19        1   0.401701
16     0.000150  0.005007      20        1   0.025904, topic_info=    Category       Freq           Term      Total  loglift  logprob
112  Default  36.000000           post  36.000000  30.0000  30.0000
241  Default  56.000000           rave  56.000000  29.0000  29.0000
180  Default  53.000000           funk  53.000000  28.0000  28.0000
304  Default  43.000000         energy  43.000000  27.0000  27.0000
94   Default  24.000000   experimental  24.000000  26.0000  26.0000
266  Default  26.000000           film  26.000000  25.0000  25.0000
99   Default  57.000000          human  57.000000  24.0000  24.0000
188  Default  45.000000          party  45.000000  23.0000  23.0000
382  Default  38.000000        electro  38.000000  22.0000  22.0000
255  Default  39.000000           tell  39.000000  21.0000  21.0000
145  Default  31.000000      last_year  31.000000  20.0000  20.0000
153  Default  34.000000   particularly  34.000000  19.0000  19.0000
190  Default  50.000000          power  50.000000  18.0000  18.0000
324  Default  22.000000         memory  22.000000  17.0000  17.0000
71   Default  47.000000           tune  47.000000  16.0000  16.0000
218  Default  39.000000           edge  39.000000  15.0000  15.0000
214  Default  23.000000        culture  23.000000  14.0000  14.0000
43   Default  45.000000          disco  45.000000  13.0000  13.0000
230  Default  46.000000           hour  46.000000  12.0000  12.0000
422  Default  30.000000        exactly  30.000000  11.0000  11.0000
312  Default  41.000000     percussion  41.000000  10.0000  10.0000
260  Default  32.000000           wave  32.000000   9.0000   9.0000
212  Default  29.000000          craft  29.000000   8.0000   8.0000
101  Default  47.000000     instrument  47.000000   7.0000   7.0000
67   Default  41.000000           star  41.000000   6.0000   6.0000
120  Default  50.000000           side  50.000000   5.0000   5.0000
105  Default  23.000000      listening  23.000000   4.0000   4.0000
25   Default  51.000000           rock  51.000000   3.0000   3.0000
354  Default  39.000000          lyric  39.000000   2.0000   2.0000
66   Default  35.000000     soundtrack  35.000000   1.0000   1.0000
..       ...        ...            ...        ...      ...      ...
331  Topic20   0.006726         subtle  16.813732   0.4346  -6.0795
2    Topic20   0.006730          array  18.088339   0.3622  -6.0788
350  Topic20   0.006731          quiet  18.111553   0.3609  -6.0788
202  Topic20   0.006739           acid  39.604790  -0.4203  -6.0776
355  Topic20   0.006737          organ  23.470678   0.1026  -6.0779
235  Topic20   0.006736            mid  26.167498  -0.0063  -6.0780
83   Topic20   0.006735          color  30.912235  -0.1730  -6.0781
189  Topic20   0.006733           pick  19.567741   0.2839  -6.0785
146  Topic20   0.006732         latter  27.182705